In [ ]:
from radas import run_experiment, get_ray_results_dir
import ray.tune as tune
from v10_1 import trainable
import torch
import pandas as pd
import numpy as np


results = await run_experiment(
    user_name="mengfan",
    experiment_name="关于比例的最终实验",
    param_space=dict(
        seed = tune.grid_search([0, 1]),  
        alpha_gen = tune.grid_search([round(x, 1) for x in np.arange(0.1, 1.01, 0.1)]),
        alpha_disc = tune.grid_search([round(x, 1) for x in np.arange(0.1, 1.01, 0.1)]),
        epochs = tune.grid_search([25]),

    ),
    # IMPORTANT: when using cluster, your trainable cannot be defined in this script, but has to be defined in a separate file, and imported in this script
    trainable=trainable,
    # When use cluster, local_storage_path needs to be specified in this way
    local_storage_path=get_ray_results_dir(),
    # Specify the cluster to use
    run_with="cluster:atol-gpu-5090",
    # Specify gram per trial
    cpu_per_trial=1,
    # If the gram_per_trial you set is larger than GRAM of a single GPU, you will be allocated with more than one GPUs
    # to fit your need. But in this case, you will need to handle how do you want to distribute your workload to these multiple GPUs
    # To check GRAM of a single GPU, use the following code:
    # from radas.clusters import gram_per_gpu
    #
    runtime_env={
        "pip": ["emoji","scikit-learn"],
        # for more runtime_env specifications, see https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#api-reference
    },
    #
    instant_gpu_allocation=1,
)

CancelledError: 

In [ ]:
results["df"]
df = results["df"]

In [ ]:
from visual_3D import plot_alpha_2d_contours
plot_alpha_2d_contours(df)